In [1]:
import ee
import ee
import geopandas as gpd
import datetime
import folium
import geemap
import datetime
import pandas as pd

In [2]:
ee.Initialize()


shapefile_path = '/home/satyukt/Sarthak/Crop_analysis/shp/AOI_02.shp'
gdf = gpd.read_file(shapefile_path)


geojson_dict = gdf.iloc[0].geometry.__geo_interface__

aoi = ee.Geometry(geojson_dict)


sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterBounds(aoi) 
bands = ["B2", "B3", "B4", "B8", "NDVI"]
def calculateNDVI(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)



In [3]:
start_date =datetime.datetime.strptime("2022-01-01", "%Y-%m-%d")

ndvi_list = []

for i in range(48):
    
    end_date = start_date + datetime.timedelta(days=14)
    filtered = sentinel2.filterDate(start_date, end_date).map(calculateNDVI).qualityMosaic('NDVI').select(bands)
    start_date = end_date
    ndvi_list.append(filtered)

In [4]:
stack_image = ee.Image(ndvi_list[0])

for i in range(1, len(ndvi_list)):
    stack_image = stack_image.addBands(ndvi_list[i])

In [5]:
crop_gdf = gpd.read_file('/home/satyukt/Sarthak/Crop_analysis/shp/crop.shp')
crop_gdf.columns = ['id', 'Class', 'geometry']

In [7]:
water_gdf = gpd.read_file("/home/satyukt/Sarthak/Crop_analysis/shp/water.shp")
water_gdf["Class"] = 4

In [8]:
classes = pd.concat([water_gdf, crop_gdf])
classes['geometry'] = classes['geometry'].centroid

/tmp/ipykernel_89365/1326186319.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  classes['geometry'] = classes['geometry'].centroid


In [9]:
classes.to_file("/home/satyukt/Sarthak/Crop_analysis/shp/classes_poits.shp")

In [10]:
train = geemap.geopandas_to_ee(classes)

In [104]:
classestraining = stack_image.sampleRegions(
    collection=train,
    properties=['Class'],
    scale=10
)
trained_classifier = ee.Classifier.smileRandomForest(100).train(
    features=classestraining,
    classProperty="Class",
    inputProperties=stack_image.bandNames(),
)
classified = stack_image.classify(trained_classifier).toInt()


In [ ]:
geemap.ee_export_image(classified,
                       filename="/home/satyukt/Desktop/classified.tif",
                       scale=10,
                       crs="EPSG:4326",
                       region=aoi)

In [82]:
Map = geemap.Map()
Map